# 🚀 Setup Milvus GPU on Windows (via WSL2 + Docker Desktop)

This section documents the full installation process to get Milvus running with GPU acceleration under **Windows + WSL2**.

---
### 0) Prereqs
- Windows 10/11 with an NVIDIA GPU
- Admin rights to install WSL2 and Docker Desktop
- Recent NVIDIA drivers installed on Windows


---
### 1) Install Docker Desktop (Windows) and Enable WSL2 Integration
1. Install **Docker Desktop** from https://www.docker.com/products/docker-desktop/
2. Open Docker Desktop → **Settings → General**:
   - ✅ Check **Use the WSL 2 based engine**
3. Go to **Settings → Resources → WSL Integration**:
   - ✅ Enable integration for your **Ubuntu** distro

> This makes the `docker` CLI inside WSL talk to Docker Desktop's engine.


---
### 2) Install WSL2 + Ubuntu (PowerShell as Administrator)
Run in **PowerShell (Admin)**:
```powershell
wsl --install -d Ubuntu
```
If you see `HCS_E_SERVICE_NOT_AVAILABLE`, **reboot** and run again.

After install:
```powershell
wsl -l -v   # should list Ubuntu with version 2
```
Launch Ubuntu from Start Menu or:
```powershell
wsl -d Ubuntu
```


---
### 3) Verify NVIDIA drivers on Windows
In **PowerShell** (Windows side):
```powershell
nvidia-smi
```
You should see your GPU listed. If not, install/update NVIDIA drivers first.


---
### 4) Inside WSL: Base packages + NVIDIA Container Toolkit
Run the following in **Ubuntu (WSL)**:
```bash
sudo apt-get update && sudo apt-get upgrade -y
sudo apt-get install -y curl wget gnupg lsb-release

# Add NVIDIA container toolkit repo (generic stable .deb list)
curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list \
  | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list

sudo apt-get update
sudo apt-get install -y nvidia-container-toolkit
```

> Note: Newer Docker Desktop releases can pass GPU into containers without installing this inside WSL, but installing it does no harm and matched what we did.

Quick GPU pass-through test from WSL:
```bash
docker run --rm --gpus all nvidia/cuda:12.2.0-base-ubuntu20.04 nvidia-smi
```
You should see your GPU.


---
### 5) Deploy Milvus Standalone (GPU) via the **official compose file**
Create a working folder and download the official GPU compose file:
```bash
mkdir -p ~/milvus && cd ~/milvus
wget https://github.com/milvus-io/milvus/releases/download/v2.6.0/milvus-standalone-docker-compose-gpu.yml -O docker-compose.yml
```

Bring it up:
```bash
docker compose up -d
docker compose ps
```

Tail logs (you should NOT see only `tini`; you should see full Milvus startup):
```bash
docker logs -f milvus-standalone
```

WebUI (optional):
```
http://127.0.0.1:9091/webui/
```

> If you only see `tini` repeatedly, you are not using the correct compose file. Use the **official** one above.


---
### 6) Python client quick test (`pymilvus`)
Install in your venv and connect:
```bash
pip install pymilvus
```

```python
from pymilvus import connections, utility
connections.connect(host="127.0.0.1", port="19530")
print("Connected to Milvus:", utility.get_server_version())
```

If you see a version string, you're good 🎉


---
# 🔴 Shutting Down Milvus Safely

When you are done using Milvus, always stop the container properly instead of just closing Ubuntu/WSL.

```bash
# Stop and remove containers, networks, and volumes defined in docker-compose.yml
docker compose down
```

> ⚠️ Always stop Milvus before shutting down Ubuntu/WSL or Windows. This prevents dangling containers and resource leaks.


---
# 🔄 Restarting Milvus GPU After Reboot (WSL2 + Docker)

Follow these steps **after rebooting your PC** to bring Milvus GPU back online.

---

### 1. Start Docker Desktop (Windows)
- Make sure Docker Desktop is running.
- Ensure **WSL2 integration** is enabled for Ubuntu:
  - Docker Desktop → Settings → Resources → WSL Integration → Enable Ubuntu.

---

### 2. Open Ubuntu (WSL2)
```bash
wsl -d Ubuntu
```

---

### 3. Check NVIDIA GPU visibility inside WSL
```bash
nvidia-smi
```
You should see your GPU listed. If not, confirm NVIDIA drivers + CUDA toolkit are installed.

---

### 4. Restart Milvus with Docker Compose
Navigate to your Milvus directory (where `docker-compose.yml` lives):
```bash
cd ~/milvus
docker compose up -d
```

---

### 5. Verify Milvus is Running
Check logs:
```bash
docker ps
docker logs -f milvus-standalone
```

You should see Milvus startup logs (not just `tini`).

---

### 6. (Optional) Reconnect From Python
```python
from pymilvus import connections
connections.connect(alias="default", uri="http://127.0.0.1:19530")
```